Classification Task - Categorical Value to Predict

Eg: Fulfillment vs Credit Default, Cancerous vs Benign Tumor, Assigning Image Category (Handwriting Recognition)

In [1]:
# !pip install autogluon

In [2]:
from autogluon.tabular import TabularDataset,TabularPredictor

c:\Users\manor\Downloads\Automated Machine Learning with AutoGluon Library in Python\autogluon_venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = TabularDataset("data/reservation/Hotel Reservations.csv")
type(data)

autogluon.core.dataset.TabularDataset

In [4]:
data.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


In [5]:
data.info()

<class 'autogluon.core.dataset.TabularDataset'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36275 non-null  object 
 1   no_of_adults                          36275 non-null  int64  
 2   no_of_children                        36275 non-null  int64  
 3   no_of_weekend_nights                  36275 non-null  int64  
 4   no_of_week_nights                     36275 non-null  int64  
 5   type_of_meal_plan                     36275 non-null  object 
 6   required_car_parking_space            36275 non-null  int64  
 7   room_type_reserved                    36275 non-null  object 
 8   lead_time                             36275 non-null  int64  
 9   arrival_year                          36275 non-null  int64  
 10  arrival_month                         36275 non-null  int64  
 11  arriv

In [6]:
# # If dataset is too big, we can take random sample
# small_df = data.sample(10000)
# small_df.info()

In [7]:
data.isnull().sum()

Booking_ID                              0
no_of_adults                            0
no_of_children                          0
no_of_weekend_nights                    0
no_of_week_nights                       0
type_of_meal_plan                       0
required_car_parking_space              0
room_type_reserved                      0
lead_time                               0
arrival_year                            0
arrival_month                           0
arrival_date                            0
market_segment_type                     0
repeated_guest                          0
no_of_previous_cancellations            0
no_of_previous_bookings_not_canceled    0
avg_price_per_room                      0
no_of_special_requests                  0
booking_status                          0
dtype: int64

In [8]:
# ALL DATA --> Split into TRAIN and TEST
# TRAIN SET to --> AUTOGLUON --> CrossValidation (tests many models)

In [9]:
len(data)

36275

In [10]:
# 0.8 * len(data)

In [11]:
# Training 30k rows and Test 6275 rows - Sort of 80-20 split

In [12]:
train_size = 30000
seed = 42

# Train Test Split using sample
train_data = data.sample(train_size, random_state=seed)
test_data = data.drop(train_data.index)

In [13]:
train_data.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
4968,INN04969,2,1,1,0,Meal Plan 1,0,Room_Type 1,3,2017,8,23,Online,0,0,0,90.00,3,Not_Canceled
34540,INN34541,2,0,1,2,Meal Plan 1,0,Room_Type 4,9,2018,2,12,Offline,0,0,0,48.67,0,Not_Canceled
36108,INN36109,2,0,2,2,Meal Plan 1,0,Room_Type 1,24,2018,12,25,Online,0,0,0,95.20,1,Not_Canceled
1553,INN01554,2,0,0,3,Meal Plan 1,0,Room_Type 1,23,2018,6,21,Online,0,0,0,127.67,0,Canceled
24974,INN24975,2,1,0,2,Meal Plan 1,0,Room_Type 4,9,2018,9,8,Online,0,0,0,201.50,2,Not_Canceled


In [14]:
test_data.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
9,INN00010,2,0,0,5,Meal Plan 1,0,Room_Type 4,44,2018,10,18,Online,0,0,0,133.44,3,Not_Canceled
16,INN00017,1,0,1,0,Meal Plan 1,0,Room_Type 1,0,2017,10,5,Offline,0,0,0,96.00,0,Not_Canceled
53,INN00054,2,0,0,4,Meal Plan 1,0,Room_Type 4,51,2017,11,11,Offline,0,0,0,60.00,1,Not_Canceled
55,INN00056,2,0,1,3,Meal Plan 1,0,Room_Type 1,100,2018,5,19,Online,0,0,0,136.00,1,Not_Canceled
60,INN00061,2,2,0,1,Meal Plan 1,1,Room_Type 6,2,2018,9,2,Online,0,0,0,258.00,1,Not_Canceled


In [15]:
train_data.info()

<class 'autogluon.core.dataset.TabularDataset'>
Int64Index: 30000 entries, 4968 to 31280
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            30000 non-null  object 
 1   no_of_adults                          30000 non-null  int64  
 2   no_of_children                        30000 non-null  int64  
 3   no_of_weekend_nights                  30000 non-null  int64  
 4   no_of_week_nights                     30000 non-null  int64  
 5   type_of_meal_plan                     30000 non-null  object 
 6   required_car_parking_space            30000 non-null  int64  
 7   room_type_reserved                    30000 non-null  object 
 8   lead_time                             30000 non-null  int64  
 9   arrival_year                          30000 non-null  int64  
 10  arrival_month                         30000 non-null  int64  
 11  ar

In [16]:
test_data.info()

<class 'autogluon.core.dataset.TabularDataset'>
Int64Index: 6275 entries, 9 to 36263
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            6275 non-null   object 
 1   no_of_adults                          6275 non-null   int64  
 2   no_of_children                        6275 non-null   int64  
 3   no_of_weekend_nights                  6275 non-null   int64  
 4   no_of_week_nights                     6275 non-null   int64  
 5   type_of_meal_plan                     6275 non-null   object 
 6   required_car_parking_space            6275 non-null   int64  
 7   room_type_reserved                    6275 non-null   object 
 8   lead_time                             6275 non-null   int64  
 9   arrival_year                          6275 non-null   int64  
 10  arrival_month                         6275 non-null   int64  
 11  arriva

Training the model

In [17]:
save_path = "reservation_predictors"

predictor = TabularPredictor(label="booking_status", path=save_path)
type(predictor)

autogluon.tabular.predictor.predictor.TabularPredictor

Report Different Metrics during fit():

    We can use **eval_metric** to provide the string of the metrics we want to evaluate predictive performance during the .fit() call of the different models. Eg:
    
        predicor.fit(train_data, label="class", eval_metric="f1")

In [18]:
predictor.fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "reservation_predictors\"
AutoGluon Version:  0.7.0
Python Version:     3.8.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22000
Train Data Rows:    30000
Train Data Columns: 18
Label Column: booking_status
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  ['Not_Canceled', 'Canceled']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = Not_Canceled, class 0 = Canceled
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive (Not_Canceled) vs negative (Canceled) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify

[1000]	valid_set's binary_error: 0.092


	0.9088	 = Validation score   (accuracy)
	1.93s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9144	 = Validation score   (accuracy)
	2.17s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9132	 = Validation score   (accuracy)
	3.18s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ...
	0.8992	 = Validation score   (accuracy)
	64.32s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9064	 = Validation score   (accuracy)
	2.16s	 = Training   runtime
	0.14s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9048	 = Validation score   (accuracy)
	3.23s	 = Training   runtime
	0.32s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.892	 = Validation score   (accuracy)
	38.47s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: XGBoost ...
	0.9052	 = Validation score   (accuracy)
	2.08s	 = Training   runtime
	0.02s	

In [19]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2     0.9192       0.776230  145.241625                0.004078           1.018211            2       True         14
1      RandomForestGini     0.9144       0.070638    2.166108                0.070638           2.166108            1       True          5
2      RandomForestEntr     0.9132       0.102415    3.175440                0.102415           3.175440            1       True          6
3         LightGBMLarge     0.9124       0.025511    1.399527                0.025511           1.399527            1       True         13
4              LightGBM     0.9088       0.038023    1.930873                0.038023           1.930873            1       True          4
5        ExtraTreesGini     0.9064       0.137434    2.162936                0.137434           2.

c:\Users\manor\Downloads\Automated Machine Learning with AutoGluon Library in Python\autogluon_venv\lib\site-packages\autogluon\core\utils\plots.py:138: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


{'model_types': {'KNeighborsUnif': 'KNNModel',
  'KNeighborsDist': 'KNNModel',
  'LightGBMXT': 'LGBModel',
  'LightGBM': 'LGBModel',
  'RandomForestGini': 'RFModel',
  'RandomForestEntr': 'RFModel',
  'CatBoost': 'CatBoostModel',
  'ExtraTreesGini': 'XTModel',
  'ExtraTreesEntr': 'XTModel',
  'NeuralNetFastAI': 'NNFastAiTabularModel',
  'XGBoost': 'XGBoostModel',
  'NeuralNetTorch': 'TabularNeuralNetTorchModel',
  'LightGBMLarge': 'LGBModel',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel'},
 'model_performance': {'KNeighborsUnif': 0.8076,
  'KNeighborsDist': 0.824,
  'LightGBMXT': 0.894,
  'LightGBM': 0.9088,
  'RandomForestGini': 0.9144,
  'RandomForestEntr': 0.9132,
  'CatBoost': 0.8992,
  'ExtraTreesGini': 0.9064,
  'ExtraTreesEntr': 0.9048,
  'NeuralNetFastAI': 0.892,
  'XGBoost': 0.9052,
  'NeuralNetTorch': 0.896,
  'LightGBMLarge': 0.9124,
  'WeightedEnsemble_L2': 0.9192},
 'model_best': 'WeightedEnsemble_L2',
 'model_paths': {'KNeighborsUnif': 'reservation_predictors\\models\\

In [20]:
# !pip install bokeh==2.0.1

In [21]:
import bokeh

In [22]:
predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2     0.9192       0.776230  145.241625                0.004078           1.018211            2       True         14
1      RandomForestGini     0.9144       0.070638    2.166108                0.070638           2.166108            1       True          5
2      RandomForestEntr     0.9132       0.102415    3.175440                0.102415           3.175440            1       True          6
3         LightGBMLarge     0.9124       0.025511    1.399527                0.025511           1.399527            1       True         13
4              LightGBM     0.9088       0.038023    1.930873                0.038023           1.930873            1       True          4
5        ExtraTreesGini     0.9064       0.137434    2.162936                0.137434           2.

c:\Users\manor\Downloads\Automated Machine Learning with AutoGluon Library in Python\autogluon_venv\lib\site-packages\autogluon\core\utils\plots.py:138: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


{'model_types': {'KNeighborsUnif': 'KNNModel',
  'KNeighborsDist': 'KNNModel',
  'LightGBMXT': 'LGBModel',
  'LightGBM': 'LGBModel',
  'RandomForestGini': 'RFModel',
  'RandomForestEntr': 'RFModel',
  'CatBoost': 'CatBoostModel',
  'ExtraTreesGini': 'XTModel',
  'ExtraTreesEntr': 'XTModel',
  'NeuralNetFastAI': 'NNFastAiTabularModel',
  'XGBoost': 'XGBoostModel',
  'NeuralNetTorch': 'TabularNeuralNetTorchModel',
  'LightGBMLarge': 'LGBModel',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel'},
 'model_performance': {'KNeighborsUnif': 0.8076,
  'KNeighborsDist': 0.824,
  'LightGBMXT': 0.894,
  'LightGBM': 0.9088,
  'RandomForestGini': 0.9144,
  'RandomForestEntr': 0.9132,
  'CatBoost': 0.8992,
  'ExtraTreesGini': 0.9064,
  'ExtraTreesEntr': 0.9048,
  'NeuralNetFastAI': 0.892,
  'XGBoost': 0.9052,
  'NeuralNetTorch': 0.896,
  'LightGBMLarge': 0.9124,
  'WeightedEnsemble_L2': 0.9192},
 'model_best': 'WeightedEnsemble_L2',
 'model_paths': {'KNeighborsUnif': 'reservation_predictors\\models\\

In [23]:
brand_new_predictor = TabularPredictor.load("reservation_predictors\\")

In [24]:
type(brand_new_predictor)

autogluon.tabular.predictor.predictor.TabularPredictor

In [25]:
test_data.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
9,INN00010,2,0,0,5,Meal Plan 1,0,Room_Type 4,44,2018,10,18,Online,0,0,0,133.44,3,Not_Canceled
16,INN00017,1,0,1,0,Meal Plan 1,0,Room_Type 1,0,2017,10,5,Offline,0,0,0,96.00,0,Not_Canceled
53,INN00054,2,0,0,4,Meal Plan 1,0,Room_Type 4,51,2017,11,11,Offline,0,0,0,60.00,1,Not_Canceled
55,INN00056,2,0,1,3,Meal Plan 1,0,Room_Type 1,100,2018,5,19,Online,0,0,0,136.00,1,Not_Canceled
60,INN00061,2,2,0,1,Meal Plan 1,1,Room_Type 6,2,2018,9,2,Online,0,0,0,258.00,1,Not_Canceled


In [26]:
y_test = test_data["booking_status"]
# X_test
test_features = test_data.drop(columns=["booking_status"])

In [27]:
# test_features

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
9,INN00010,2,0,0,5,Meal Plan 1,0,Room_Type 4,44,2018,10,18,Online,0,0,0,133.44,3
16,INN00017,1,0,1,0,Meal Plan 1,0,Room_Type 1,0,2017,10,5,Offline,0,0,0,96.00,0
53,INN00054,2,0,0,4,Meal Plan 1,0,Room_Type 4,51,2017,11,11,Offline,0,0,0,60.00,1
55,INN00056,2,0,1,3,Meal Plan 1,0,Room_Type 1,100,2018,5,19,Online,0,0,0,136.00,1
60,INN00061,2,2,0,1,Meal Plan 1,1,Room_Type 6,2,2018,9,2,Online,0,0,0,258.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36241,INN36242,2,0,0,2,Meal Plan 2,0,Room_Type 1,256,2018,6,15,Offline,0,0,0,115.00,1
36242,INN36243,1,0,2,4,Meal Plan 1,0,Room_Type 1,63,2017,9,24,Offline,0,0,0,118.43,0
36258,INN36259,2,0,2,2,Meal Plan 1,0,Room_Type 4,40,2018,8,13,Online,0,0,0,149.40,2
36260,INN36261,3,0,1,4,Meal Plan 1,1,Room_Type 4,178,2018,8,1,Online,0,0,0,137.25,1


In [28]:
y_preds = brand_new_predictor.predict(test_features)
y_preds

9        Not_Canceled
16       Not_Canceled
53       Not_Canceled
55       Not_Canceled
60       Not_Canceled
             ...     
36241        Canceled
36242    Not_Canceled
36258    Not_Canceled
36260        Canceled
36263    Not_Canceled
Name: booking_status, Length: 6275, dtype: object

In [29]:
# Manually
sum(y_test == y_preds)/len(y_test)

0.9005577689243028

In [33]:
metrics = brand_new_predictor.evaluate_predictions(y_true=y_test,
                                        y_pred=y_preds,
                                        auxiliary_metrics=True)
metrics

Evaluation: accuracy on test data: 0.9005577689243028
Evaluations on test data:
{
    "accuracy": 0.9005577689243028,
    "balanced_accuracy": 0.8765545945532529,
    "mcc": 0.7684857902939071,
    "f1": 0.928011075219197,
    "precision": 0.9130533484676504,
    "recall": 0.9434670419892095
}


{'accuracy': 0.9005577689243028,
 'balanced_accuracy': 0.8765545945532529,
 'mcc': 0.7684857902939071,
 'f1': 0.928011075219197,
 'precision': 0.9130533484676504,
 'recall': 0.9434670419892095}

### Inference on a new datapoint.
Assuming that a customer just booked a room in your hotel, is it likely that the reservation will be cancelled?

In [34]:
reservation = {
    "Booking_ID" : "INN01961234234",
    "no_of_adults" : 1,
    "no_of_children" : 0,
    "no_of_weekend_nights" : 0,
    "no_of_week_nights" : 3,
    "type_of_meal_plan" : "Meal Plan 1",
    "required_car_parking_space" : 0,
    "room_type_reserved" : "Room_Type 1",
    "lead_time" : 190,
    "arrival_year" : 2023,
    "arrival_month" : 11,
    "arrival_date" : 3,
    "market_segment_type" : "Online",
    "repeated_guest" : 0,
    "no_of_previous_cancellations" : 0,
    "no_of_previous_bookings_not_canceled" : 10,
    "avg_price_per_room" : 190.9,
    "no_of_special_requests" : 0
}

To predict on new, unseen data, we need to pass a TabularDataset, a pandas DataFrame or the path to the data

In [35]:
# for a single row dataset, the dictionary needs to be encapsulated in a list
reservation_data = TabularDataset.from_dict([reservation])

In [37]:
type(reservation_data)

autogluon.core.dataset.TabularDataset

In [36]:
brand_new_predictor.predict(reservation_data)

0    Canceled
Name: booking_status, dtype: object

Looks like the customer will cancel its reservation

Classification Metrics:
- Accuracy: (TP+TN)/Total. *How often is the model correct?*
- Precision: (TP)/Total Predicted Positives. *When prediction is positive, how often is it correct?*
- Recall (a.k.a sensitivity): (TP)/ Total Actual Positives. *When it actually is a positive case, how often is it correct? Another way of describing what recall is, is the ability to answer the question how many relevant cases are found?*
- Other Metrics reported by AutoGluon
    - F1-Score: = (2*precision*recall)/(precision+recall). Defined as the harmonic mean of the precision and recall.
    - Balanced Accuracy
    - MCC: Matthews Coefficient

In F1 Score we use the harmonic mean (instead of the normal mean) as it allows the entire harmonic mean to go to *zero* if **either** precision or recall ends up being zero

Depending on the model, there is typically a trade-off between precision and recall, which can be explored using the ROC curve.

Accuracy Paradox!!
- Any classifier dealing with **imbalanced** classes has to confront the issue of the accuracy paradox.
- **Imbalanced** classes will always result in a distored accuracy reflecting better performance than what is truly warranted.

Note that recall and precision do not fall to the accuracy paradox.

Due to imbalanced datasets, this means we shouldn't solely rely on accuracy as a metric. Which is also where other performance metrics come into play, such as Precision, Recall and F1 Score. 

### Interpretability

An important aspect of machine learning system is to identify the importance of the individual features.
By calling **predictor.feature_importance(train_data)**, we obtain an importance score, paired with std, p_value and the 99th percentile for each feature.<br />
Note, that the label is required for this to work, thus we use train_data

For more information on how the feature importance is calculated check out these two ressource:


https://explained.ai/rf-importance/ <br />
https://scikit-learn.org/stable/modules/permutation_importance.html#id2



In [38]:
# for this we need FEATURES and LABEL
brand_new_predictor.feature_importance(test_data)

These features in provided data are not utilized by the predictor and will be ignored: ['Booking_ID']
Computing feature importance via permutation shuffling for 17 features using 5000 rows with 5 shuffle sets...
	169.84s	= Expected runtime (33.97s per shuffle set)
	61.19s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
lead_time,0.18180,0.003056,9.579346e-09,5,0.188093,0.175507
no_of_special_requests,0.13260,0.005136,2.695805e-07,5,0.143175,0.122025
market_segment_type,0.08140,0.001860,3.269893e-08,5,0.085230,0.077570
avg_price_per_room,0.07552,0.002759,2.133823e-07,5,0.081201,0.069839
arrival_month,0.03840,0.003797,1.132779e-05,5,0.046219,0.030581
arrival_year,0.02284,0.000829,2.083581e-07,5,0.024548,0.021132
no_of_weekend_nights,0.01916,0.002496,3.377038e-05,5,0.024298,0.014022
no_of_adults,0.01424,0.002080,5.314429e-05,5,0.018524,0.009956
arrival_date,0.01064,0.001381,3.333302e-05,5,0.013484,0.007796
type_of_meal_plan,0.00904,0.001410,6.876754e-05,5,0.011943,0.006137


From above, immediately know that the lead time is the most important feature and that kind of makes sense.
If you have a super long lead time, you're probably more likely to cancel. 
Or if you have a super short lead time, you're very much less likely to cancel.

Let's only change lead_time to 100

In [39]:
reservation = {
    "Booking_ID" : "INN01961234234",
    "no_of_adults" : 1,
    "no_of_children" : 0,
    "no_of_weekend_nights" : 0,
    "no_of_week_nights" : 3,
    "type_of_meal_plan" : "Meal Plan 1",
    "required_car_parking_space" : 0,
    "room_type_reserved" : "Room_Type 1",
    "lead_time" : 100,
    "arrival_year" : 2023,
    "arrival_month" : 11,
    "arrival_date" : 3,
    "market_segment_type" : "Online",
    "repeated_guest" : 0,
    "no_of_previous_cancellations" : 0,
    "no_of_previous_bookings_not_canceled" : 10,
    "avg_price_per_room" : 190.9,
    "no_of_special_requests" : 0
}

reservation_data = TabularDataset.from_dict([reservation])
brand_new_predictor.predict(reservation_data)

0    Not_Canceled
Name: booking_status, dtype: object